<a href="https://colab.research.google.com/github/gladcolor/pdf_tabel_to_csv/blob/master/PDF_tabel_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [1]:
pip install tabula-py

     |████████████████████████████████| 11.7MB 4.6MB/s 


# Import packages

In [4]:
import glob 
import os
import pandas as pd

import tabula



from zipfile import ZipFile
from urllib.request import urlopen
from natsort import natsorted


# Download pdfs

In [5]:
file_url = r'https://github.com/gladcolor/pdf_tabel_to_csv/raw/master/pdfs.zip'

zipresp = urlopen(file_url)
basename = os.path.basename(file_url)
tempzip = open(basename, "wb")
tempzip.write(zipresp.read())
tempzip.close()

zf = ZipFile(basename)
zf.extractall(path = ".")
zf.close()

pdfs = glob.glob('pdfs/*.pdf')
pdfs = natsorted(pdfs)
print(f'Found {len(pdfs)} PDF files: {pdfs}')

Found 184 PDF files: ['pdfs/Abbeville.pdf', 'pdfs/Abbeville_2014-2016.pdf', 'pdfs/Abbeville_2015-2017.pdf', 'pdfs/Abbeville_2016-2018.pdf', 'pdfs/Aiken.pdf', 'pdfs/Aiken_2014-2016.pdf', 'pdfs/Aiken_2015-2017.pdf', 'pdfs/Aiken_2016-2018.pdf', 'pdfs/Allendale.pdf', 'pdfs/Allendale_2014-2016.pdf', 'pdfs/Allendale_2015-2017.pdf', 'pdfs/Allendale_2016-2018.pdf', 'pdfs/Anderson.pdf', 'pdfs/Anderson_2014-2016.pdf', 'pdfs/Anderson_2015-2017.pdf', 'pdfs/Anderson_2016-2018.pdf', 'pdfs/Bamberg.pdf', 'pdfs/Bamberg_2014-2016.pdf', 'pdfs/Bamberg_2015-2017.pdf', 'pdfs/Bamberg_2016-2018.pdf', 'pdfs/Barnwell.pdf', 'pdfs/Barnwell_2014-2016.pdf', 'pdfs/Barnwell_2015-2017.pdf', 'pdfs/Barnwell_2016-2018.pdf', 'pdfs/Beaufort.pdf', 'pdfs/Beaufort_2014-2016.pdf', 'pdfs/Beaufort_2015-2017.pdf', 'pdfs/Beaufort_2016-2018.pdf', 'pdfs/Berkeley.pdf', 'pdfs/Berkeley_2014-2016.pdf', 'pdfs/Berkeley_2015-2017.pdf', 'pdfs/Berkeley_2016-2018.pdf', 'pdfs/Calhoun.pdf', 'pdfs/Calhoun_2014-2016.pdf', 'pdfs/Calhoun_2015-2017.

Get template CSV.

Please ignore the year shown in column "MEASURE". Column "YEAR" is correct.

In [6]:
template_csv_url = r'https://raw.githubusercontent.com/gladcolor/pdf_tabel_to_csv/master/template.csv'
temp_df = pd.read_csv(template_csv_url)
temp_df

,COUNTY,YEAR,Indicator,MEASURE,COUNT,VALUE,RANK,STATE
0,NaN,NaN,Births,Births with expected payor Medicaid (percent ...,NaN,NaN,NaN,NaN
1,NaN,NaN,Births,Breastfeeding initiation (percent of all live ...,NaN,NaN,NaN,NaN
2,NaN,NaN,Births,Low birthweight births (<2500 grams; percent o...,NaN,NaN,NaN,NaN
3,NaN,NaN,Births,Mothers receiving adequate prenatal care (perc...,NaN,NaN,NaN,NaN
4,NaN,NaN,Births,Mothers who smoked during pregnancy (percent o...,NaN,NaN,NaN,NaN
5,NaN,NaN,Births,Preterm births (<37 weeks gestation; percent o...,NaN,NaN,NaN,NaN
6,NaN,NaN,Births,Teen live births (rate per 1000 female populat...,NaN,NaN,NaN,NaN
7,NaN,NaN,Infant Mortality,Infant mortality (rate per 1000 live births; 2...,NaN,NaN,NaN,NaN
8,NaN,NaN,Chronic Diseases; Risk Factors; and Health Beh...,Coronary heart disease (percent; 2013-2015),NaN,NaN,NaN,NaN
9,NaN,NaN,Chronic Diseases; Risk Factors; and Health Beh...,Stroke (percent; 2013-2015),NaN,NaN,NaN,NaN


# Extract tables.

Note: `tabula` misses read the last two rows of 2015 PDFs, and misses read the lat row of 2018 PDFs.

In [7]:
def process_pdfs():
    save_dir = './csv'
    all_dfs = []
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for idx, pdf in enumerate(pdfs[:]):
        basename = os.path.basename(pdf)[:-4]
        county_name = basename.split("_")[0]
        df = tabula.read_pdf(pdf, pages = 1, lattice = False)[0]
        # Is year 2015?
        year = basename[-4:]
        if not year.isnumeric():
            year = "2015"
            new_df = process_2015_pdf(df)
        else:
            new_df = process_2018_pdf(df)

        print("Processing file #", idx + 1, pdf)        
        new_df.iloc[:, 0] = county_name
        new_df.iloc[:, 1] = year
        new_name = os.path.join(save_dir, basename + ".csv")

        new_df.to_csv(new_name, index=False)   
        
        all_dfs.append(new_df)

    print("Concatenating all CSV files...")

    all_df = pd.concat(all_dfs, axis=0)
    new_name = os.path.join(save_dir,  "all_counties.csv")
    all_df.to_csv(new_name, index=False)   
    print("Saved CSV as: ", new_name)
    print("Finished.")    

    return pd.read_csv(new_name)

def process_2015_pdf(df):
    new_df = temp_df.copy()   
    new_df['COUNT'] = "NA"     
    new_df.iloc[:40, 5:8] = df.iloc[1:41, 2:5].to_numpy()  # note here!    
    new_df.iloc[40:42, 5:8] = df.iloc[41:43, 1:4].to_numpy()
    
    return new_df

def process_2018_pdf(df):        
    new_df = temp_df.copy()        
    new_df.iloc[:39, 4:] = df.iloc[1:40, 2:6].to_numpy()  # note here!
    new_df.iloc[39:40, 4:] = 'NA'
    new_df.iloc[40:41, 4:] = df.iloc[40:41, 2:6].to_numpy()
    new_df.iloc[41:42, 4:] = df.iloc[41:42, 1:5].to_numpy()   
    return new_df

process_pdfs()



Processing file # 1 pdfs/Abbeville.pdf
Processing file # 2 pdfs/Abbeville_2014-2016.pdf
Processing file # 3 pdfs/Abbeville_2015-2017.pdf
Processing file # 4 pdfs/Abbeville_2016-2018.pdf
Processing file # 5 pdfs/Aiken.pdf
Processing file # 6 pdfs/Aiken_2014-2016.pdf
Processing file # 7 pdfs/Aiken_2015-2017.pdf
Processing file # 8 pdfs/Aiken_2016-2018.pdf
Processing file # 9 pdfs/Allendale.pdf
Processing file # 10 pdfs/Allendale_2014-2016.pdf
Processing file # 11 pdfs/Allendale_2015-2017.pdf
Processing file # 12 pdfs/Allendale_2016-2018.pdf
Processing file # 13 pdfs/Anderson.pdf
Processing file # 14 pdfs/Anderson_2014-2016.pdf
Processing file # 15 pdfs/Anderson_2015-2017.pdf
Processing file # 16 pdfs/Anderson_2016-2018.pdf
Processing file # 17 pdfs/Bamberg.pdf
Processing file # 18 pdfs/Bamberg_2014-2016.pdf
Processing file # 19 pdfs/Bamberg_2015-2017.pdf
Processing file # 20 pdfs/Bamberg_2016-2018.pdf
Processing file # 21 pdfs/Barnwell.pdf
Processing file # 22 pdfs/Barnwell_2014-2016.pdf

,COUNTY,YEAR,Indicator,MEASURE,COUNT,VALUE,RANK,STATE
0,Abbeville,2015,Births,Births with expected payor Medicaid (percent ...,NaN,58.0,NaN,51.2
1,Abbeville,2015,Births,Breastfeeding initiation (percent of all live ...,NaN,60.6,26,72.4
2,Abbeville,2015,Births,Low birthweight births (<2500 grams; percent o...,NaN,8.6,10,9.6
3,Abbeville,2015,Births,Mothers receiving adequate prenatal care (perc...,NaN,82.2,2,74.3
4,Abbeville,2015,Births,Mothers who smoked during pregnancy (percent o...,NaN,17.3,40,10.6
...,...,...,...,...,...,...,...,...
7723,York,2018,Health Care Access,Population insured by private health insurance...,NaN,3.3,NaN,5.2
7724,York,2018,Health Care Access,Population without health insurance (percent; ...,NaN,8.7,NaN,11.0
7725,York,2018,Home and Environmental Hazards,Crude rate of carbon monoxide (CO) exposures r...,NaN,NaN,NaN,NaN
7726,York,2018,Home and Environmental Hazards,Elevated (≥5 mcg/dL) blood lead tests in child...,NaN,1.2,11,1.8


In [ ]:
! rm -f -r ./csv